# Entity Relationship Diagram (ERD)

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# Libraries

In [1]:
import numpy as np
import pandas as pd
import gc
import time
# https://stackoverflow.com/questions/3693771/trying-to-understand-python-with-statement-and-context-managers
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Helper Functions

## Timer

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield # runs the with block here
    print('{} - done in {:.0f}s'.format(title, time.time() - t0))

## One-Hot Encoder

In [3]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## Preprocessing of application train test

In [4]:
def application_train_test(num_rows = None, nan_as_category=False):
    # read and union data
    DIR = '~/kaggle-home-credit'
    df = pd.read_csv(DIR + '/input/application_train.csv', nrows=num_rows)
    test_df = pd.read_csv(DIR + '/input/application_test.csv', nrows=num_rows)
    print('Train shape: {}'.format(df.shape))
    print('Test shape: {}'.format(test_df.shape))
    
    # Union of train and test sets
    df = df.append(test_df).reset_index()
    
    # remove rows without gender
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # identify columns of client provided documents
    docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
    #print(docs)
    
    # identify columns of real estate and personal documents
    live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
    
    # replace 365243 as NaN
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # this returns a series of median rather than a dataframe for mapping
    inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
    
    # Feature Engineering
    df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    # can consider sum instead of kurtosis
    df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
    df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
    # added 1 to avoid division by zero
    df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
    # maps median of median income by organization
    df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
    df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
    df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
    df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    df['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    
    # Binary encoding of categorical binary features
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        # returns labels and uniques
        df[bin_feature], uniques = pd.factorize(df[bin_feature])

    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # FLAG_DOCUMENT_3 is important hence did not drop it
    dropcolum=['FLAG_DOCUMENT_2','FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 
    'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']
    df= df.drop(dropcolum,axis=1)
    del test_df
    gc.collect()
    
    print('Combined shape: {}'.format(df.shape))
    
    return df


## Preprocess Bureau and Balance 

In [5]:
def bureau_and_balance(num_rows = 10000, nan_as_category = True):
    DIR = '~/kaggle-home-credit'
    bureau = pd.read_csv(DIR + '/input/bureau.csv', nrows = num_rows)
    bb = pd.read_csv(DIR + '/input/bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': [ 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': [ 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': [ 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

        

In [6]:
bureau_and_balance()

,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_SUM,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100053,-2287.714286,2.215852e+05,-1519.333333,-1137.571429,0.0,NaN,9.265564e+04,6.485895e+05,0.000000e+00,0.000,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100568,-493.857143,1.367698e+05,110.500000,-244.857143,0.0,8178.88500,1.192699e+05,8.348891e+05,6.037136e+04,422599.500,...,0.0,0.0,0.0,0.0,0.00,0.0,NaN,NaN,NaN,0.0
100653,-1237.571429,9.864580e+05,-295.333333,-536.285714,0.0,NaN,1.247832e+05,8.734824e+05,6.525306e+04,456771.420,...,0.0,0.0,0.0,0.0,0.00,0.0,NaN,NaN,NaN,0.0
100802,-525.000000,NaN,928.000000,-25.000000,0.0,NaN,1.575000e+05,1.575000e+05,1.576305e+05,157630.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100819,-917.555556,3.330211e+05,-305.055556,-487.888889,0.0,NaN,2.321152e+05,4.178074e+06,1.276136e+05,1403749.890,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
100915,-362.000000,NaN,1031.000000,-48.000000,49.0,NaN,1.350000e+05,1.350000e+05,4.500000e+00,4.500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101060,-1328.500000,7.790070e+05,-522.500000,-820.750000,0.0,0.00000,2.041439e+05,8.165755e+05,1.022186e+05,408874.320,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
101133,-2385.000000,NaN,-2079.000000,-1536.000000,0.0,15454.17000,7.506000e+04,7.506000e+04,0.000000e+00,0.000,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
101562,-1228.285714,1.223359e+05,-24.400000,-141.857143,0.0,16665.94800,4.217733e+05,2.952413e+06,2.129252e+05,1490476.500,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
